In [1]:
# CO2 und Energiedaten Datenset Vorbereitung 
import pandas as pd

# Our World in Data CO2 & Energy Dataset laden
# Quelle: https://github.com/owid/co2-data
df_owid = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

print(f"OWID CO2 & Energy Dataset geladen: {df_owid.shape}")

# Zeitraum auf Analysezeitraum begrenzen und nur Länder mit ISO-Codes
df_owid = df_owid[
    (df_owid['year'] >= 1960) &
    (df_owid['year'] <= 2018) &
    (df_owid['iso_code'].notna())
]

print(f"Nach Filterung (ab 1960, mit ISO-Code): {df_owid.shape}")
print(f"Zeitraum: {df_owid['year'].min()} - {df_owid['year'].max()}")
print(f"Länder: {df_owid['iso_code'].nunique()}")

# Relevante Variablen für CO2-Analyse auswählen
selected_columns = {
    # Identifikation
    'country': 'Country',
    'iso_code': 'ISO3', 
    'year': 'Year',
    
    # Bevölkerung und Wirtschaft
    'population': 'Population',
    'gdp': 'GDP',
    
    # CO2-Emissionen (gesamt und nach Quellen)
    'co2': 'CO2_total',
    'co2_per_capita': 'CO2_per_capita',
    'coal_co2': 'CO2_coal',
    'oil_co2': 'CO2_oil', 
    'gas_co2': 'CO2_gas',
    'cement_co2': 'CO2_cement',
    'flaring_co2': 'CO2_flaring',
    'other_industry_co2': 'CO2_other',
    
    # Energieverbrauch und Intensität
    'primary_energy_consumption': 'Primary_energy',
    'energy_per_capita': 'Energy_per_capita',
    'energy_per_gdp': 'Energy_intensity',
    
    # CO2-Intensität
    'co2_per_gdp': 'CO2_intensity',
    'co2_per_unit_energy': 'CO2_per_energy',
    'co2_including_luc_per_gdp': 'CO2_intensity_luc',
    'consumption_co2_per_gdp': 'CO2_intensity_consumption'
}

# Dataset erstellen
df_co2_energy = df_owid[list(selected_columns.keys())].rename(columns=selected_columns)

# Daten sortieren
df_co2_energy = df_co2_energy.sort_values(['Country', 'Year']).reset_index(drop=True)

# Datenqualität bewerten
print(f"\nDATENQUALITÄT ")
print(f"Endgültige Datenpunkte: {len(df_co2_energy)}")
print(f"Länder: {df_co2_energy['Country'].nunique()}")

# Intensität
print(f"\nINTENSITÄTS FEATURES ")
intensity_vars = ['Energy_intensity', 'CO2_intensity', 'CO2_per_energy']
for var in intensity_vars:
    if var in df_co2_energy.columns:
        completeness = (df_co2_energy[var].notna().sum() / len(df_co2_energy)) * 100
        stats = df_co2_energy[var].describe()
        print(f"{var}: {completeness:.1f}% verfügbar, Median: {stats['50%']:.3f}")

# Vollständigkeit der Hauptvariablen
key_vars = ['CO2_total', 'CO2_per_capita', 'Primary_energy', 'Population', 'GDP']
print(f"\nVollständigkeit Hauptvariablen:")
for var in key_vars:
    if var in df_co2_energy.columns:
        completeness = (df_co2_energy[var].notna().sum() / len(df_co2_energy)) * 100
        print(f"  {var}: {completeness:.1f}%")

# Fehlende Werte in CO2-Unterkategorien
co2_types = ['CO2_coal', 'CO2_oil', 'CO2_gas', 'CO2_cement', 'CO2_flaring', 'CO2_other']
missing_co2_details = df_co2_energy[co2_types].isnull().sum()
if missing_co2_details.sum() > 0:
    print(f"\nFehlende Werte CO2-Unterkategorien:")
    for var, missing in missing_co2_details.items():
        if missing > 0:
            pct = (missing / len(df_co2_energy)) * 100
            print(f"  {var}: {missing} ({pct:.1f}%)")

# Dataset speichern
df_co2_energy.to_csv('../data/processed/co2_energy_data.csv', index=False)
print(f"\nDatei gespeichert: co2_energy_data.csv")

# Stichprobe anzeigen
display(df_co2_energy.head())

OWID CO2 & Energy Dataset geladen: (50191, 79)
Nach Filterung (ab 1960, mit ISO-Code): (12862, 79)
Zeitraum: 1960 - 2018
Länder: 218

DATENQUALITÄT 
Endgültige Datenpunkte: 12862
Länder: 218

INTENSITÄTS FEATURES 
Energy_intensity: 55.3% verfügbar, Median: 1.319
CO2_intensity: 71.8% verfügbar, Median: 0.260
CO2_per_energy: 66.7% verfügbar, Median: 0.214

Vollständigkeit Hauptvariablen:
  CO2_total: 94.6%
  CO2_per_capita: 94.1%
  Primary_energy: 67.4%
  Population: 99.1%
  GDP: 72.6%

Fehlende Werte CO2-Unterkategorien:
  CO2_coal: 4655 (36.2%)
  CO2_oil: 769 (6.0%)
  CO2_gas: 6497 (50.5%)
  CO2_cement: 1046 (8.1%)
  CO2_flaring: 5092 (39.6%)
  CO2_other: 11457 (89.1%)

Datei gespeichert: co2_energy_data.csv


,Country,ISO3,Year,Population,GDP,CO2_total,CO2_per_capita,CO2_coal,CO2_oil,CO2_gas,CO2_cement,CO2_flaring,CO2_other,Primary_energy,Energy_per_capita,Energy_intensity,CO2_intensity,CO2_per_energy,CO2_intensity_luc,CO2_intensity_consumption
0,Afghanistan,AFG,1960,9035048.0,1.303325e+10,0.414,0.046,0.127,0.269,0.0,0.018,NaN,NaN,NaN,NaN,NaN,0.032,NaN,1.370,NaN
1,Afghanistan,AFG,1961,9214082.0,1.314629e+10,0.491,0.053,0.176,0.293,0.0,0.022,NaN,NaN,NaN,NaN,NaN,0.037,NaN,1.163,NaN
2,Afghanistan,AFG,1962,9404411.0,1.336763e+10,0.689,0.073,0.297,0.363,0.0,0.029,NaN,NaN,NaN,NaN,NaN,0.052,NaN,1.052,NaN
3,Afghanistan,AFG,1963,9604491.0,1.363030e+10,0.707,0.074,0.264,0.392,0.0,0.051,NaN,NaN,NaN,NaN,NaN,0.052,NaN,0.990,NaN
4,Afghanistan,AFG,1964,9814318.0,1.387050e+10,0.839,0.085,0.300,0.476,0.0,0.062,NaN,NaN,NaN,NaN,NaN,0.060,NaN,0.785,NaN


In [2]:
# Wirtschaftsindikatoren Datenset Vorbereitung
import pandas as pd

# World Development Indicators (WDI) laden
# Quelle: https://www.kaggle.com/datasets/theworldbank/world-development-indicators
df_wdi = pd.read_csv('../data/raw/WDIData.csv')

print(f"World Development Indicators geladen: {df_wdi.shape}")

# Relevante Wirtschaftsindikatoren für CO2-Analyse auswählen
economic_indicators = {
    'NY.GDP.MKTP.CD': 'GDP_current_USD',           # BIP (nominal)
    'NY.GDP.PCAP.CD': 'GDP_per_capita_USD',        # BIP pro Kopf
    'NY.GDP.MKTP.KD.ZG': 'GDP_growth_annual',      # BIP-Wachstum (%)
    'SP.POP.TOTL': 'Population_total',             # Gesamtbevölkerung
    'SP.URB.TOTL.IN.ZS': 'Urban_population_pct',   # Urbanisierungsgrad (%)
    'NV.IND.TOTL.ZS': 'Industry_share_GDP',        # Industrieanteil am BIP (%)
    'NV.MNF.TOTL.ZS': 'Manufacturing_share_GDP',   # Verarbeitendes Gewerbe (%)
    'NV.SRV.TOTL.ZS': 'Services_share_GDP',        # Dienstleistungssektor (%)
    'NE.TRD.GNFS.ZS': 'Trade_share_GDP',           # Außenhandel (% des BIP)
    'EN.ATM.CO2E.PC': 'CO2_per_capita_WDI'         # CO2 pro Kopf (zur Validierung)
}

# Daten nach ausgewählten Indikatoren filtern
df_wdi_filtered = df_wdi[df_wdi['Indicator Code'].isin(economic_indicators.keys())].copy()
print(f"Nach Indikator-Filterung: {df_wdi_filtered.shape}")

# Datenstruktur von Wide- zu Long-Format transformieren
year_columns = [col for col in df_wdi_filtered.columns if col.isdigit()]
print(f"Verfügbare Jahre: {year_columns[0]} bis {year_columns[-1]} ({len(year_columns)} Jahre)")

df_wdi_long = pd.melt(
    df_wdi_filtered,
    id_vars=['Country Name', 'Country Code', 'Indicator Code'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Jahr als Integer und Zeitfilter anwenden
df_wdi_long['Year'] = df_wdi_long['Year'].astype(int)
df_wdi_long = df_wdi_long[df_wdi_long['Year'] >= 1960]
(df_wdi_long['Year'] <= 2018)
print(f"Nach Zeitfilter (ab 1960): {df_wdi_long.shape}")

# Von Long- zu Wide-Format für finale Struktur
df_wdi_pivot = df_wdi_long.pivot_table(
    index=['Country Code', 'Country Name', 'Year'],
    columns='Indicator Code',
    values='Value'
).reset_index()

# Spalten umbenennen und bereinigen
df_wdi_pivot.columns.name = None
df_wdi_pivot = df_wdi_pivot.rename(columns=economic_indicators)

# Konsistente Spaltennamen für Merge
df_wdi_pivot = df_wdi_pivot.rename(columns={
    'Country Code': 'ISO3',
    'Country Name': 'Country'
})

# Weltbank-Aggregate entfernen (nur echte Länder behalten)
regional_aggregates = [
    # Regionale Aggregate
    'ARB', 'CSS', 'CEB', 'EAP', 'EAS', 'ECA', 'ECS', 'EMU', 'EUU', 'HIC', 
    'LAC', 'LCN', 'LIC', 'LMC', 'LMY', 'MEA', 'MIC', 'MNA', 'NAC', 'OED', 
    'SAS', 'SSA', 'SSF', 'TEA', 'TEC', 'TLA', 'TMN', 'TSA', 'TSS', 'UMC', 'WLD',
    
    # Demografische und Entwicklungsaggregate
    'PST', 'PSS', 'EAP', 'EDU', 'PRE', 'ECD', 'LCD', 'PCD',
    
    # Weltbank Institutionelle Aggregate
    'IBD', 'IBT', 'IDB', 'IDX'
]

# Zusätzlich: Aggregate über Ländernamen entfernen (falls ISO3-Codes fehlen)
aggregate_name_patterns = [
    'demographic dividend', 'IDA', 'IBRD', 'World', 'income',
    'developed', 'developing', 'Sub-Saharan', 'East Asia', 'Latin America',
    'Middle East', 'North Africa', 'South Asia', 'Europe', 'Central Asia',
    'Caribbean', 'Pacific', 'OECD', 'EU', 'Euro'
]

# Erst ISO3-basierte Filterung
df_economic = df_wdi_pivot[~df_wdi_pivot['ISO3'].isin(regional_aggregates)].copy()
print(f"Nach Entfernung regionaler Aggregate (ISO3): {df_economic.shape}")

# Dann namebasierte Filterung für übersehene Aggregate
for pattern in aggregate_name_patterns:
    mask = df_economic['Country'].str.contains(pattern, case=False, na=False)
    removed_count = mask.sum()
    if removed_count > 0:
        print(f"Entferne {removed_count} Aggregate mit '{pattern}' im Namen")
        df_economic = df_economic[~mask]

print(f"Nach vollständiger Aggregat-Entfernung: {df_economic.shape}")

# Spalten strukturieren
base_columns = ['Country', 'ISO3', 'Year']
indicator_columns = [col for col in df_economic.columns if col not in base_columns]
df_economic = df_economic[base_columns + sorted(indicator_columns)]

# Daten sortieren
df_economic = df_economic.sort_values(['Country', 'Year']).reset_index(drop=True)

# Datenqualitätsanalyse
print(f"\nDATENQUALITÄT")
print(f"Zeitraum: {df_economic['Year'].min()} - {df_economic['Year'].max()}")
print(f"Länder: {df_economic['Country'].nunique()}")
print(f"Endgültige Datenpunkte: {len(df_economic)}")

# Vollständigkeit pro Indikator
print(f"\nVollständigkeit der Wirtschaftsindikatoren:")
for col in indicator_columns:
    if col in df_economic.columns:
        completeness = (df_economic[col].notna().sum() / len(df_economic)) * 100
        print(f"  {col}: {completeness:.1f}%")

# Identifikation kritischer Datenlücken
critical_vars = ['GDP_current_USD', 'Population_total', 'Urban_population_pct']
print(f"\nKritische Variablen (für Modellierung):")
for var in critical_vars:
    if var in df_economic.columns:
        missing = df_economic[var].isnull().sum()
        total = len(df_economic)
        print(f"  {var}: {total - missing}/{total} verfügbar ({((total - missing)/total)*100:.1f}%)")

# Dataset speichern
df_economic.to_csv('../data/processed/economic_indicators.csv', index=False)
print(f"\nDatei gespeichert: economic_indicators.csv")

# Stichprobe anzeigen
display(df_economic.head())

World Development Indicators geladen: (422136, 64)
Nach Indikator-Filterung: (2376, 64)
Verfügbare Jahre: 1960 bis 2018 (59 Jahre)
Nach Zeitfilter (ab 1960): (140184, 5)
Nach Entfernung regionaler Aggregate (ISO3): (12985, 12)
Entferne 116 Aggregate mit 'demographic dividend' im Namen
Entferne 116 Aggregate mit 'IDA' im Namen
Entferne 58 Aggregate mit 'developed' im Namen
Nach vollständiger Aggregat-Entfernung: (12695, 12)

DATENQUALITÄT
Zeitraum: 1960 - 2018
Länder: 220
Endgültige Datenpunkte: 12695

Vollständigkeit der Wirtschaftsindikatoren:
  CO2_per_capita_WDI: 78.9%
  Trade_share_GDP: 66.8%
  Industry_share_GDP: 59.1%
  Services_share_GDP: 53.9%
  GDP_current_USD: 75.4%
  GDP_growth_annual: 72.1%
  GDP_per_capita_USD: 75.3%
  Population_total: 99.7%
  Urban_population_pct: 99.1%

Kritische Variablen (für Modellierung):
  GDP_current_USD: 9566/12695 verfügbar (75.4%)
  Population_total: 12653/12695 verfügbar (99.7%)
  Urban_population_pct: 12578/12695 verfügbar (99.1%)

Datei gesp

,Country,ISO3,Year,CO2_per_capita_WDI,GDP_current_USD,GDP_growth_annual,GDP_per_capita_USD,Industry_share_GDP,Population_total,Services_share_GDP,Trade_share_GDP,Urban_population_pct
0,Afghanistan,AFG,1960,0.046060,5.377778e+08,NaN,59.777327,NaN,8996351.0,NaN,11.157027,8.401
1,Afghanistan,AFG,1961,0.053604,5.488889e+08,NaN,59.878153,NaN,9166764.0,NaN,12.550610,8.684
2,Afghanistan,AFG,1962,0.073765,5.466667e+08,NaN,58.492874,NaN,9345868.0,NaN,14.227644,8.976
3,Afghanistan,AFG,1963,0.074233,7.511112e+08,NaN,78.782758,NaN,9533954.0,NaN,26.035511,9.276
4,Afghanistan,AFG,1964,0.086292,8.000000e+08,NaN,82.208444,NaN,9731361.0,NaN,26.944448,9.586
